In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"D:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 1000)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 98.11it/s]


Total cost = 45014.850539202125

Cluster 347 assigned to Class 7. Cost: 128.88416952465232
Cluster 227 assigned to Class 16. Cost: 129.52809137945562
Cluster 245 assigned to Class 18. Cost: 126.24064633472572
Cluster 144 assigned to Class 22. Cost: 112.27586684004405
Cluster 225 assigned to Class 23. Cost: 106.6689201171879
Cluster 104 assigned to Class 51. Cost: 131.82697168895632
Cluster 174 assigned to Class 87. Cost: 148.28667641250436
Cluster 218 assigned to Class 88. Cost: 156.35406655911805
Cluster 32 assigned to Class 89. Cost: 112.43383307995985
Cluster 99 assigned to Class 93. Cost: 95.09907902016356
Cluster 122 assigned to Class 95. Cost: 124.82738312779408
Cluster 342 assigned to Class 96. Cost: 88.75979244248737
Cluster 162 assigned to Class 105. Cost: 100.34848856123891
Cluster 244 assigned to Class 112. Cost: 120.89051857151672
Cluster 199 assigned to Class 117. Cost: 96.65112515953022
Cluster 191 assigned to Class 121. Cost: 120.66140871743518
Cluster 365 assigned to Cl

In [6]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
